    16_230113_140_bus_display
    
         data : k_bus_info.csv,20210823_socar_zone.csv
      save : 
       display : bus_info.html

In [73]:
#1. package load
import requests
import json
import os
import math
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import folium
import matplotlib.pyplot as plt
import seaborn as sns

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

import warnings
warnings.filterwarnings(action='ignore') 

print(pd.__version__)
print(gpd.__version__)
print(folium.__version__)

1.3.3
0.10.1
0.12.1


In [74]:
#2. 프로젝트(1)  ... data load
#path = os.getenv('HOME') + '/aiffel/socar_gis/data/20210823_socar_zone.csv'
path = '/aiffel/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/004_node_data/20210823_socar_zone.csv'

socar_zone_info_df = pd.read_csv(path)
socar_zone_info_df.head()

,zone_name,lng,lat,region1,region2,region3
0,비전2 파출소사거리,127.102939,36.995989,경기도,평택시,비전동
1,수원시청역 9번출구,127.032953,37.262378,경기도,수원시 팔달구,인계동
2,목포 상동현대아파트 앞,126.415655,34.811289,전라남도,목포시,상동
3,단국대 산학협력관 앞,127.166626,36.832965,충청남도,천안시 동남구,신부동
4,목원대학교 앞,127.337858,36.329906,대전광역시,서구,도안동


In [75]:
k_socar_zone_df = socar_zone_info_df[socar_zone_info_df['region1'] == '경기도']

In [76]:
# mg1 = MarkerCluster()
# for _, row in k_socar_zone_df.iterrows():
#     mg1.add_child(    
#         folium.Marker(location = [row['lat'], row['lng']],
#                popup=row['zone_name'],
#                icon=folium.Icon(color='blue',icon='star')
#               )
#     )

    2. 프로젝트(1) 서울시 쏘카존 데이터를 활용한 시각화
       1) data load : socar zone data
       2) 시각화
          - base nap : 서울시 중심부 위도, 경도
          - MarkerCluster 객체 
          - map draw

In [77]:
#3. 프로젝트(2) ... 2) 버스data load  
path = os.getenv('HOME') + '/aiffel/000_aiffelthon_230111/3김연수/002_data_dir/004_node_data/k_bus_info.csv'
bus_line_info_df =  pd.read_csv(path)
bus_line_info_df.head()

,정류장번호,정류장명,위도,경도,정보수집일시,모바일단축번호,도시코드,도시명,관리도시명
0,GGB176500010,광주유스퀘어(종합터미널),35.160517,126.881217,2.022100e+13,0,31250,광주시,경기도
1,GGB179501240,광주비아정류소,35.225400,126.826317,2.022100e+13,0,31250,광주시,경기도
2,GGB200000001,하광교느티나무,37.317600,127.016900,2.022100e+13,1165,31010,수원시,경기도
3,GGB200000002,하광교,37.315933,127.019167,2.022100e+13,1173,31010,수원시,경기도
4,GGB200000003,비석거리입구,37.313233,127.021017,2.022100e+13,1180,31010,수원시,경기도


In [80]:
#3. 프로젝트(2) ... 2) 버스data 전처리 .. 컬럼명 변경
bus_line_info_df.rename(
    columns={
        "위도": "lat",
        "경도": "lng",
    },
    inplace=True
)

bus_line_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36879 entries, 0 to 36878
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   정류장번호    36879 non-null  object 
 1   정류장명     36879 non-null  object 
 2   lat      36879 non-null  float64
 3   lng      36879 non-null  float64
 4   정보수집일시   36879 non-null  float64
 5   모바일단축번호  36879 non-null  int64  
 6   도시코드     36879 non-null  int64  
 7   도시명      36879 non-null  object 
 8   관리도시명    36879 non-null  object 
dtypes: float64(3), int64(2), object(4)
memory usage: 2.5+ MB


In [81]:
#3. 프로젝트(2) ... 2) 버스 정류장 위치데이터 리스트 만들기
bus_station_location_data = [[row["lat"], row["lng"]] for _, row in bus_line_info_df.iterrows()]
print('슝~')
#bus_station_location_data

슝~


In [82]:
#3. 프로젝트(2) ... 3)시각화  ... # Folium 초기 Map 객체 생성 

center = [37.541, 126.986]

map = folium.Map(
    location=center,
    zoom_start= 15
)
print('슝~')

슝~


In [83]:
# #4. 프로젝트(3)  .. # 2)레이어 적용을 위한 그룹 만들기
# # 대표 그룹 만들기
fg = folium.FeatureGroup(name="groups")
map.add_child(fg)

# 버스정류장 그룹 만들기
bus = plugins.FeatureGroupSubGroup(fg, "bus_group")
map.add_child(bus)

# 쏘카 그룹 만들기
socar = plugins.FeatureGroupSubGroup(fg, "socar_zone")
map.add_child(socar)

# 레이어 콘트롤 박스 만들기
folium.LayerControl(collapsed=False).add_to(map)

In [84]:
#3. 프로젝트(2) ... 3) 시각화 ... # 버스정류장 밀도
plugins.HeatMap(bus_station_location_data).add_to(bus)

In [85]:
#4. 프로젝트(3)  ..3) 쏘카존 마커 클러스터 만들기
socar_zone_data = [[row["lat"], row["lng"]] for _, row in k_socar_zone_df.iterrows()]
popups = [k_socar_zone_df.loc[idx, "zone_name"] for idx, row in k_socar_zone_df.iterrows()] #popup시 표시 될 주소

mg = plugins.MarkerCluster( socar_zone_data, popups=popups,control=False).add_to(socar)

# map.add_child(mg)

In [86]:
#3. 프로젝트(2) ... 3) 시각화 ... # 생성한 맵 확인
map
map.add_child(mg)

In [87]:
map.save("/aiffel/aiffel/000_aiffelthon_230111/3김연수/003_map_xml/bus_info.html")